In [0]:
#挂载到谷歌云盘，本地跑不用挂载
from google.colab import drive
drive.mount('/content/drive')
!pip install keras-transformer
!pwd
!ls "/content/drive/My Drive/Colab Notebooks/middle_data/"

In [1]:
import numpy as np
import pickle
import operator
from keras_transformer import get_model, decode
# main_path = '/content/drive/My Drive/Colab Notebooks/'    #Google Colab FilePath
# path = main_path + 'middle_data/'
path = 'middle_data/'
with open(path + 'encode_input.pkl', 'rb') as f:
    encode_input = pickle.load(f)
with open(path + 'decode_input.pkl', 'rb') as f:
    decode_input = pickle.load(f)
with open(path + 'decode_output.pkl', 'rb') as f:
    decode_output = pickle.load(f)
with open(path + 'source_token_dict.pkl', 'rb') as f:
    source_token_dict = pickle.load(f)
with open(path + 'target_token_dict.pkl', 'rb') as f:
    target_token_dict = pickle.load(f)
with open(path + 'source_tokens.pkl', 'rb') as f:
    source_tokens = pickle.load(f)
print('Done')

Done


In [ ]:
print(len(source_token_dict))
print(len(target_token_dict))
print(len(encode_input))
# 构建模型
model = get_model(
    token_num=max(len(source_token_dict), len(target_token_dict)),
    embed_dim=64,
    encoder_num=2,
    decoder_num=2,
    head_num=4,
    hidden_dim=256,
    dropout_rate=0.05,
    use_same_embed=False,  # 不同语言需要使用不同的词嵌入
)
model.compile('adam', 'sparse_categorical_crossentropy')
# model.summary()
print('Done')

In [ ]:
#训练模型
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
filepath = path + "models/W-" + "-{epoch:3d}-{loss:.4f}-.h5"
checkpoint = ModelCheckpoint(filepath,
                    monitor='loss',
                    verbose=1,
                    save_best_only=True,
                    mode='min',
                    save_freq='epoch',
                    )
reduce_lr = ReduceLROnPlateau(monitor='loss', 
                    factor=0.2, 
                    patience=2, 
                    verbose=1, 
                    mode='min', 
                    min_delta=0.0001, 
                    cooldown=0, 
                    min_lr=0
                    )
callbacks_list = [checkpoint, reduce_lr]
model.fit(
    x=[np.array(encode_input[:20000]), np.array(decode_input[:20000])],
    y=np.array(decode_output[:20000]),
    epochs=100,
    batch_size=64, 
    verbose=1,
    callbacks=callbacks_list, 
    # class_weight=None, 
    # max_queue_size=5, 
#    workers=1, 
#    use_multiprocessing=False,
    # shuffle=False,
#    initial_epoch=initial_epoch_
    )
# model.save(main_path+'modles/model.h5')a

In [ ]:
#加载模型
model.load_weights('model\W-- 99-0.0070-.h5')
target_token_dict_inv = {v: k for k, v in target_token_dict.items()}
print('Done')

In [10]:
from keras.preprocessing import sequence
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import jieba
import requests

def get_input(seq):
    seq = ' '.join(jieba.lcut(seq, cut_all=False))
    # seq = ' '.join(seq)
    seq = seq.split(' ')
    print(seq)
    seq = ['<START>'] + seq + ['<END>']
    seq = seq + ['<PAD>'] * (34 - len(seq))
    print(seq)
    for x in seq:
        try:
            source_token_dict[x]
        except KeyError:
            flag=False
            break
        else:
            flag=True
    if(flag):
        seq = [source_token_dict[x] for x in seq]
    return flag, seq
def get_ans(seq):
    decoded = decode(
    model,
    [seq],
    start_token=target_token_dict['<START>'],
    end_token=target_token_dict['<END>'],
    pad_token=target_token_dict['<PAD>'],
    # top_k=8,
    # temperature=1.0,
  )
    print(' '.join(map(lambda x: target_token_dict_inv[x], decoded[0][1:-1])))

while True:
    seq = input()
    if seq == 'x':
        break
    flag, seq = get_input(seq)
    if(flag):
        get_ans(seq)
    else:
        print('听不懂呢。')

['']
['<START>', '', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
1/1 [==============================] - 0s 28ms/step
That's .
['']
['<START>', '', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
1/1 [==============================] - 0s 35ms/step
That's .


KeyboardInterrupt: Interrupted by user